In [1]:
import pandas as pd
import numpy as np

from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

import warnings

In [2]:
from sklearn.ensemble import BaggingClassifier

In [3]:
warnings.filterwarnings('ignore')

In [4]:
dtClassifier = tree.DecisionTreeClassifier()
knnClassifier = KNeighborsClassifier()
rfClassifier = RandomForestClassifier()
svmClassifier = SVC(probability=True)
mlpClassifier = MLPClassifier()

In [7]:
enade_todos_anos = pd.read_csv("../../teste_OHE_curso_treinamento.csv")

In [10]:
k = 5

array_folds = np.array_split(enade_todos_anos, k)

for i_fold in range(k):
    
    # Separação dos folds
    
    folds_treinamento = array_folds.copy()
    fold_teste = array_folds[i_fold]
    
    del folds_treinamento[i_fold]
    
    folds_treinamento = pd.concat(folds_treinamento, sort=False)
    
    # Separação em X e y
    
    numero_caracteristicas = enade_todos_anos.shape[1] - 1
    
    X_folds_treinamento = folds_treinamento.iloc[:, 0:numero_caracteristicas]
    y_folds_treinamento = folds_treinamento.iloc[:, -1]
    
    X_fold_teste = fold_teste.iloc[:, 0:numero_caracteristicas]
    y_fold_teste = fold_teste.iloc[:, -1]
    
    # Normalização de treino e teste
    
    normalizador_treinamento = StandardScaler()
    
    normalizador_treinamento.fit(X_folds_treinamento)
    treinamento_normalizado = normalizador_treinamento.transform(X_folds_treinamento)
    
    normalizador_teste = StandardScaler()
    
    normalizador_teste.fit(X_fold_teste)
    teste_normalizado = normalizador_teste.transform(X_fold_teste)
    
    parametros_para_busca = {"C": [0.01, 0.1, 1, 10, 100], 
                              "kernel": ["linear", "poly", "rbf"],
                              "gamma": [0.1, 1, 10],
                              "tol": [0.1, 0.001, 0.00001]}
    
    buscaParam = HalvingGridSearchCV(svmClassifier, parametros_para_busca, scoring="f1_weighted").fit(treinamento_normalizado, y_folds_treinamento)
    
    baggingClassifier = BaggingClassifier(buscaParam.best_estimator_)
    baggingClassifier.fit(treinamento_normalizado, y_folds_treinamento)
    
    y_predito = baggingClassifier.predict(teste_normalizado)
    
    resultado = f1_score(y_fold_teste, y_predito, average='weighted')
    
    # print(f"Teste: fold {i_fold}")
    # print(f"Classificador: \n\n{melhor_classificador}\n")
    print(f"{resultado} ", end="")

0.5096477646393963 0.47095918725074065 0.4567084547090056 0.43593542176789374 0.43775497449438955 